In [1]:
import scda
import os
import shutil
import numpy as np

import matplotlib.pyplot as plt
%pylab inline --no-import-all
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
matplotlib.rcParams['image.cmap'] = 'gray'
matplotlib.rcParams['axes.linewidth'] = 1.
matplotlib.rcParams['lines.linewidth'] = 2.5
matplotlib.rcParams['font.size'] = 12

import logging
#scda.configure_log()

Populating the interactive namespace from numpy and matplotlib


In [2]:
os.getcwd()

'/Users/ntz/SCDA/scda_pytools/examples'

# Single APLC design evaluation, without LS alignment tolerance

## Set the design parameters

In [3]:
pupil_params = {'N': 500, 'prim':'hex3', 'centobs':True, 'secobs':'X', 'thick':'025'}
fpm_params = {'rad': 4., 'M':60}
ls_params_noaligntol = {'obscure':0, 'id':25, 'od':80, 'aligntol':None}
#ls_params_noaligntol = {'obscure':1, 'id':25, 'od':83, 'spad':5, 'aligntol':None}
image_params = {'c': 10., 'ida':3.5, 'oda':10., 'bw':0., 'Nlam':1, 'fpres':2}

In [4]:
#work_dir = "."
work_dir = "/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/"
if not os.path.exists(os.path.normpath(work_dir)):
    os.mkdir(work_dir)
#input_dir = "./InputMasks" # location of input TelAp, FPM, and LS arrays
input_dir = "/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/InputMasks/"
TelAp_dir = os.path.join(input_dir, "TelAp")
FPM_dir = os.path.join(input_dir, "FPM")
LS_dir = os.path.join(input_dir, "LS")

In [5]:
design_params_noaligntol = {'Pupil': pupil_params, 'FPM': fpm_params,
                            'LS': ls_params_noaligntol, 'Image': image_params}
fileorg = {'work dir': work_dir, 'TelAp dir': TelAp_dir,
           'FPM dir': FPM_dir, 'LS dir': LS_dir}
bar_solver = {'method': 'bar'}
barhom_solver = {'method': 'barhom', 'convtol': 9.}
#dualsimp_solver = {'method': 'dualsimp'}
#barhom_solver = {'method': 'barhom'}

## Initiate the coronagraph object

In [6]:
hexap_coron = scda.QuarterplaneAPLC(design=design_params_noaligntol, fileorg=fileorg, solver=barhom_solver)

## Show the file organization

In [7]:
hexap_coron.fileorg

{'FPM dir': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/InputMasks/FPM',
 'FPM fname': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/InputMasks/FPM/FPM_quart_occspot_M060.dat',
 'LS dir': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/InputMasks/LS',
 'LS fname': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/InputMasks/LS/LS_quart_ann25D80_clear_N0500.dat',
 'TelAp dir': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/InputMasks/TelAp',
 'TelAp fname': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/InputMasks/TelAp/TelAp_quart_hex3X025cobs1_N0500.dat',
 'ampl src dir': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/',
 'ampl src fname': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/APLC_quart_hex3X025cobs1_N0500_FPM400M060_LSann25D80clear_Img100C_75DA100_BW00Nlam01fpres2_linbarhompre1convtol90.mod',
 'eval dir': '/astro/opticslab1/SCDA/Scripts/AMPL/scda_qpaplc_tests/',
 'exec script dir': '/astro/opticslab1/SCDA/Scrip

### Check the status of input files needed to run the AMPL program

In [8]:
hexap_coron.check_ampl_input_files()
print("All the input files for AMPL are in place? {0:}".format(hexap_coron.ampl_infile_status))

All the input files for AMPL are in place? False


## Write the AMPL source file

In [9]:
hexap_coron.write_ampl(override_infile_status=False, overwrite=True)

ERROR:root:Error: the most recent input file check for this design configuration failed.
ERROR:root:The override_infile_status switch is off, so write_ampl() will now abort.
ERROR:root:See previous warnings in the log to see what file was missing during the initialization


## Create a bundled source + input file subdirectory for both designs

In [ ]:
bundled_dir = os.path.join(hexap_coron.fileorg['work dir'], "qpaplc_ampl_bundle")
#bundled_coron_list = scda.make_ampl_bundle([hexap_coron_noaligntol, hexap_coron_aligntol], bundled_dir)
#os.listdir(bundled_dir)

## Evaluate basic coronagraph metrics

In [ ]:
hexap_coron.get_metrics()

# Full-plane on-axis PSF evaluation

### Reset input file locations, in case out of STScI network

In [ ]:
#hexap_coron.fileorg['TelAp fname'] = os.path.join(os.path.expanduser("~/Box Sync/scda/InputMasks/TelAp"),
#                                                  os.path.basename(hexap_coron.fileorg['TelAp fname']))

#hexap_coron.fileorg['FPM fname'] = os.path.join(os.path.expanduser("~/Box Sync/scda/InputMasks/FPM"),
#                                                os.path.basename(hexap_coron.fileorg['FPM fname']))

#hexap_coron.fileorg['LS fname'] = os.path.join(os.path.expanduser("~/Box Sync/scda/InputMasks/LS"),
#                                               os.path.basename(hexap_coron.fileorg['LS fname']))

### Load and reorganize mask arrays

In [ ]:
TelAp_qp = np.loadtxt(hexap_coron.fileorg['TelAp fname'])
TelAp = np.concatenate((np.concatenate((TelAp_qp[::-1,::-1], TelAp_qp[:,::-1]),axis=0),
                        np.concatenate((TelAp_qp[::-1,:], TelAp_qp),axis=0)), axis=1)

FPM_qp = np.loadtxt(hexap_coron.fileorg['FPM fname'])
FPM = np.concatenate((np.concatenate((FPM_qp[::-1,::-1], FPM_qp[:,::-1]),axis=0),
                      np.concatenate((FPM_qp[::-1,:], FPM_qp),axis=0)), axis=1)

LS_qp = np.loadtxt(hexap_coron.fileorg['LS fname'])
LS = np.concatenate((np.concatenate((LS_qp[::-1,::-1], LS_qp[:,::-1]),axis=0),
                     np.concatenate((LS_qp[::-1,:], LS_qp),axis=0)), axis=1)

#new_sol_fname = os.path.join(hexap_coron.fileorg['sol dir'],
#                             "ApodSol_APLC_quart_hex3X025cobs1_N0250_FPM400M050_LSann25D85clear_Img100C_35DA080_BW10Nlam03fpres2_linbarhompre1convtol2e9.dat")
#                             "ApodSol_APLC_quart_hex3X025cobs1_N0125_FPM400M050_LSann25D85clear_Img100C_35DA080_BW10Nlam03fpres2_linbarhompre1_convtol1e9.dat")
#                             "ApodSol_APLC_quart_hex3X025cobs1_N0125_FPM400M050_LSann25D85clear_Img100C_35DA080_BW10Nlam03fpres2_linbarpre1_ftol.dat")
#hexap_coron.fileorg['sol fname'] = new_sol_fname
#hexap_coron.design['Image']['bw'] = 0.10

A_col = np.loadtxt(hexap_coron.fileorg['sol fname'])[:,-1]
A_qp = A_col.reshape(TelAp_qp.shape)
A = np.concatenate((np.concatenate((A_qp[::-1,::-1], A_qp[:,::-1]),axis=0),
                    np.concatenate((A_qp[::-1,:], A_qp),axis=0)), axis=1)

plt.figure(figsize=(8,8))
plt.imshow(A*TelAp)

In [ ]:
psf_img_polychrom, radial_seps, radial_psf_profile_polychrom = hexap_coron.eval_onax_psf(Nlam=1)

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(np.log10(psf_img_polychrom[-2]), vmin=-11, vmax=-7, cmap='CMRmap')
plt.colorbar()

## Radial PSF profile

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(radial_seps, np.log10(radial_psf_profile_polychrom[0,:]))
plt.plot(radial_seps, np.log10(radial_psf_profile_polychrom[radial_psf_profile_polychrom.shape[0]/2,:]))
plt.plot(radial_seps, np.log10(radial_psf_profile_polychrom[-1,:]))
plt.legend(['blue','center','red'], loc='upper left')
plt.xlabel('angular sep. (lambda_0/D)')
plt.ylabel('log10(I/I0)')

In [ ]:
radial_psf_profile_polychrom[-1,0]